# Portfolio Blog

# Traditional Chinese Calligraphy Recognition Using CNN
This is the portfolio blog for INFO 7390 Spring instructed by Prof. Nik Brown.<br>By Houze Liu: [liu.hou@husky.neu.edu](mailto:liu.hou@husky.neu.edu) / <a href="https://github.com/HouHouHouHouHouHou">github</a>

## Introduction
This blog is about to recognize 40 different Traditional Chinese Characters(TCC) using Convolutional Neural Network(CNN).

In this blog, we will implement different CNN on our dataset and evaluate their performance by accuracy on classification. The plateform we will use is PyTorch, which is easier to learn compared to TensorFlow.

Convolutional Neural Networks(CNN) work by moving small filters across an input image. One filter can be regarded as a reception. It acts like a sight neure which 'sees' one part of input image one time, and can get activated accordingly. This makes Convolutional Netwroks very powerful on capturing patterns, and thus can keep more complex information than simple multilayer perceptrons(MLP). The differenc between CNN and MLP is that CNN has convolution layers. Besides, MLP has the layers same with the CNN's rest layers (we call them fully connected layers, or FC layers in terms of CNN). 

![Convolution Layer1](images/image3.jpeg)
![Convolution Layer2](images/image2.jpeg)
The numer of filters we use to convolve the input image euqals the number of 'stacks' of the output activation maps. That gave us a new input with more channels and usually smaller size. 
![Convolution Layer3](images/image1.jpeg)

![](/images/image1)

import content

Load data

pytorch

performance